<a href="https://colab.research.google.com/github/blackstone-555/Stock-Market-Sentiment-Analysis/blob/main/Stock_Market_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Market Sentiment Analysis**

In [265]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

**Loading Textual(News) data and stock prices data**

Downloaded textual (news) data from https://bit.ly36fFPI6

In [272]:
df1 = pd.read_csv("/content/india-news-headlines.csv (1).crdownload")

Downloaded historical stock prices from (https://finance.yahoo.com)

In [274]:
df2 = pd.read_csv("/content/^BSESN.csv")

**Data pre-processing**

In [276]:
df1.head()

,publish_date,headline_category,headline_text
0,20010101,sports.wwe,win over cena satisfying but defeating underta...
1,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
2,20010102,unknown,Fissures in Hurriyat over Pak visit
3,20010102,unknown,America's unwanted heading for India?
4,20010102,unknown,For bigwigs; it is destination Goa


In [277]:
df1['publish_date'] = np.array([str(str(str(x)[:4]) + '/' +str(str(x)[4:6]) + '/' + str(str(x)[6:])) for x in df1['publish_date']]) 

In [278]:
df1.head()

,publish_date,headline_category,headline_text
0,2001/01/01,sports.wwe,win over cena satisfying but defeating underta...
1,2001/01/02,unknown,Status quo will not be disturbed at Ayodhya; s...
2,2001/01/02,unknown,Fissures in Hurriyat over Pak visit
3,2001/01/02,unknown,America's unwanted heading for India?
4,2001/01/02,unknown,For bigwigs; it is destination Goa


In [279]:
df1['publish_date'] = pd.to_datetime(df1['publish_date'])

In [280]:
df1 = df1[['publish_date', 'headline_text']]

In [281]:
df1.columns = ['publish_date','headline']

**Sequencing news having same date**

In [282]:
dict_news ={}
t1 = df1.loc[0,'publish_date']
t2 = str(df1.loc[0 , 'headline'])
for x in range(1,len(df1)):
  if df1.loc[x , 'publish_date'] == t1:
    t2 += '. '+str(df1.loc[x,'headline'])
  else:
    dict_news[df1.loc[x-1,'publish_date']] = t2
    t2 = ""
    t1 = df1.loc[x ,'publish_date']  

In [283]:
indexes = np.arange(0,len(dict_news))

In [284]:
df_news = pd.DataFrame(indexes)
df_news['publish_date'] = dict_news.keys()

In [285]:
df_news.head()

,0,publish_date
0,0,2001-01-01
1,1,2001-01-02
2,2,2001-01-03
3,3,2001-01-04
4,4,2001-01-05


In [286]:
l = []
for i in dict_news.keys():
  l.append(dict_news[i])

In [287]:
l[0]

'win over cena satisfying but defeating undertaker bigger roman reigns'

In [288]:
df_news['headline'] = np.array(l)

In [289]:
del df_news[0]

**Sentiment analysis**

In [290]:
from textblob import TextBlob
polarity = []
subjectivity=[]
tuples=[]
for i in df_news['headline'].values:
  my_valence=TextBlob(i)
  tuples.append(my_valence.sentiment)

In [291]:
pip install -U git+https://github.com/sloria/TextBlob.git@dev

  Cloning https://github.com/sloria/TextBlob.git (to revision dev) to /tmp/pip-req-build-pirmj7om
  Running command git clone -q https://github.com/sloria/TextBlob.git /tmp/pip-req-build-pirmj7om
  Created wheel for textblob: filename=textblob-0.17.0-py2.py3-none-any.whl size=636707 sha256=1316f240757ae23c6567afd54a0d5bbb9d34ad188dbd3c7d9d19d31c7bc67e54
  Stored in directory: /tmp/pip-ephem-wheel-cache-27fpof2v/wheels/77/e7/4b/f2ac6cb423f94d4dccb473aefd022eae48c70b793eb4c53a5d
Successfully built textblob
  Found existing installation: textblob 0.17.0
    Uninstalling textblob-0.17.0:
      Successfully uninstalled textblob-0.17.0


In [292]:
for i in tuples:
  polarity.append(i[0])
  subjectivity.append(i[1])

In [293]:
df_news['polarity']=np.array(polarity)
df_news['subjectivity']=np.array(subjectivity)


In [294]:
temp =['positive','negative','neutral']
temp1 = ['factual','public']
polarity = []
subjectivity=[]
for i in range(len(df_news)):
  pol = df_news.iloc[i]['polarity']
  sub = df_news.iloc[i]['subjectivity']
  if pol>=0:
   if pol>=0.2:
     polarity.append(temp[0])
   else:
     polarity.append(temp[2])
  else:
    if pol<= -0.2:
      polarity.append(temp[1])
    else:
      polarity.append(temp[2])  
  if sub >=0.4:
    subjectivity.append(temp1[1])
  else:
    subjectivity.append(temp1[0])

In [295]:
df_news['sentiment'] = polarity
df_news['opinion'] = subjectivity

In [296]:
df_news.shape

(2494, 6)

In [297]:
df1.shape

(522825, 2)

In [298]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   publish_date  2494 non-null   datetime64[ns]
 1   headline      2494 non-null   object        
 2   polarity      2494 non-null   float64       
 3   subjectivity  2494 non-null   float64       
 4   sentiment     2494 non-null   object        
 5   opinion       2494 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 117.0+ KB


In [299]:
df2['hl_pct'] = ((df2['High']-df2['Low'])/df2['Low'])*100
df2['pct_change'] = ((df2['Close']-df2['Open'])/df2['Open'])*100

In [300]:
df2.head()

,Date,Open,High,Low,Close,Adj Close,Volume,hl_pct,pct_change
0,2001-01-02,3953.219971,4028.570068,3929.370117,4018.879883,4018.879883,0.0,2.524576,1.660922
1,2001-01-03,3977.580078,4067.659912,3977.580078,4060.020020,4060.020020,0.0,2.264689,2.072616
2,2001-01-04,4180.970215,4180.970215,4109.549805,4115.370117,4115.370117,0.0,1.737913,-1.569016
3,2001-01-05,4116.339844,4195.009766,4115.350098,4183.729980,4183.729980,0.0,1.935672,1.637137
4,2001-01-08,4164.759766,4206.720215,4101.529785,4120.430176,4120.430176,0.0,2.564663,-1.064397


In [301]:
df2 = df2.drop(['Open','High','Close','Low'],axis =1)

In [302]:
df2.head()

,Date,Adj Close,Volume,hl_pct,pct_change
0,2001-01-02,4018.879883,0.0,2.524576,1.660922
1,2001-01-03,4060.020020,0.0,2.264689,2.072616
2,2001-01-04,4115.370117,0.0,1.737913,-1.569016
3,2001-01-05,4183.729980,0.0,1.935672,1.637137
4,2001-01-08,4120.430176,0.0,2.564663,-1.064397


In [303]:
df2.tail()

,Date,Adj Close,Volume,hl_pct,pct_change
2770,2011-12-26,15970.750000,11800.0,1.505349,1.194893
2771,2011-12-27,15873.950195,10800.0,1.579089,-0.688378
2772,2011-12-28,15727.849609,11800.0,1.412826,-0.797956
2773,2011-12-29,15543.929688,12800.0,1.348071,-0.734473
2774,2011-12-30,15454.919922,12800.0,1.863578,-0.825421


In [304]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [305]:
df2.head()

,Date,Adj Close,Volume,hl_pct,pct_change
0,2001-01-02,4018.879883,0.0,2.524576,1.660922
1,2001-01-03,4060.020020,0.0,2.264689,2.072616
2,2001-01-04,4115.370117,0.0,1.737913,-1.569016
3,2001-01-05,4183.729980,0.0,1.935672,1.637137
4,2001-01-08,4120.430176,0.0,2.564663,-1.064397


In [306]:
df2 = df2.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2717 entries, 0 to 2774
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        2717 non-null   datetime64[ns]
 1   Adj Close   2717 non-null   float64       
 2   Volume      2717 non-null   float64       
 3   hl_pct      2717 non-null   float64       
 4   pct_change  2717 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 127.4 KB


In [307]:
df_news.columns= ['Date','Headline','Polarity','Subjectivity','sentiment','opinion']

In [308]:
df_news.columns

Index(['Date', 'Headline', 'Polarity', 'Subjectivity', 'sentiment', 'opinion'], dtype='object')

**Merging of News and stock price dataset on basis of Dates**

In [309]:
mdf = df_news.merge(df2,how='inner',on ='Date',left_index=True)
mdf.head()

,Date,Headline,Polarity,Subjectivity,sentiment,opinion,Adj Close,Volume,hl_pct,pct_change
0,2001-01-02,. Fissures in Hurriyat over Pak visit. America...,0.151333,0.282333,neutral,factual,4018.879883,0.0,2.524576,1.660922
1,2001-01-03,. Think again; Laxman tells Sangh Parivar. Hur...,0.163636,0.363636,neutral,factual,4060.020020,0.0,2.264689,2.072616
2,2001-01-04,. Vajpayee gives big push to tech education; R...,0.087961,0.446847,neutral,public,4115.370117,0.0,1.737913,-1.569016
3,2001-01-05,. Hawkings' day out. Light combat craft takes ...,0.260401,0.473984,positive,public,4183.729980,0.0,1.935672,1.637137
4,2001-01-08,. Frontrunner for key US post seen having Pak ...,-0.019214,0.414653,neutral,public,4120.430176,0.0,2.564663,-1.064397


In [310]:
reqdf  = mdf[['Date','Polarity','Subjectivity','Adj Close','hl_pct','pct_change']]
reqdf.head()

,Date,Polarity,Subjectivity,Adj Close,hl_pct,pct_change
0,2001-01-02,0.151333,0.282333,4018.879883,2.524576,1.660922
1,2001-01-03,0.163636,0.363636,4060.020020,2.264689,2.072616
2,2001-01-04,0.087961,0.446847,4115.370117,1.737913,-1.569016
3,2001-01-05,0.260401,0.473984,4183.729980,1.935672,1.637137
4,2001-01-08,-0.019214,0.414653,4120.430176,2.564663,-1.064397


In [311]:
reqdf = reqdf.set_index('Date')

**Feature scaling using MinMaxScaler**

In [312]:
from sklearn.preprocessing import MinMaxScaler

In [313]:
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(reqdf))
df.columns = reqdf.columns
df.index  = reqdf.index

In [314]:
df.head()

,Polarity,Subjectivity,Adj Close,hl_pct,pct_change
Date,,,,,
2001-01-02,0.524545,0.498235,0.081643,0.115688,0.681352
2001-01-03,0.541322,0.641711,0.084011,0.101539,0.704859
2001-01-04,0.438129,0.788554,0.087196,0.072859,0.496922
2001-01-05,0.673274,0.836442,0.091130,0.083626,0.679993
2001-01-08,0.291981,0.731740,0.087487,0.117870,0.525736


In [315]:
train = df[:int(0.7*(len(df)))]
test =  df[int(0.7*(len(df))):]

In [316]:
from statsmodels.tsa.vector_ar.var_model import VAR
model = VAR(endog =train)
fitted = model.fit()

**Predicted vs Test**

In [317]:
pred = fitted.forecast(fitted.y, steps=len(test))
predicted = pd.DataFrame(pred,columns=df.columns)


In [318]:
predicted.head()

,Polarity,Subjectivity,Adj Close,hl_pct,pct_change
0,0.414728,0.744575,0.341928,0.072977,0.581895
1,0.414388,0.729295,0.342504,0.072462,0.583726
2,0.414003,0.726323,0.343076,0.072302,0.583699
3,0.413923,0.725737,0.343645,0.072264,0.583669
4,0.413921,0.725643,0.344215,0.072256,0.583661


In [319]:
test.head()

,Polarity,Subjectivity,Adj Close,hl_pct,pct_change
Date,,,,,
2005-11-23,0.441913,0.836070,0.347472,0.049791,0.650500
2005-11-24,0.380178,0.708086,0.353555,0.047794,0.645163
2005-11-25,0.425792,0.597576,0.359837,0.042394,0.646808
2005-11-28,0.396004,0.633611,0.367993,0.039344,0.643744
2005-11-29,0.406859,0.635238,0.364323,0.054811,0.542428


In [320]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(predicted['Adj Close'],test['Adj Close']))

0.0916673571080565

In [325]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print ('R Squared =',r2_score(test['Adj Close'],predicted['Adj Close']))
print ('MAE =',mean_absolute_error(test['Adj Close'],predicted['Adj Close']))
print ('MSE =',mean_squared_error(test['Adj Close'],predicted['Adj Close']))

R Squared = 0.6550771722083735
MAE = 0.07343497530882377
MSE = 0.008402904359175957
